# Data Scrubbing and Cleaning for Improved Analysis with DataCleaner  

Check my Github for [this project's details](https://github.com/westrany/Data-Scrubbing-and-Cleaning-for-Improved-Analysis-with-DataCleaner/blob/main/README.md). More info on the chosen dataset has been [uploaded to Kaggle](https://www.kaggle.com/datasets/mariafitas/goodreads-small-dataset).

---  
## Increasing Data Rate Limit  
When working on this project, I've often gotten the "IOPub data rate exceeded" message. To resolve this, I have increased the notebook's data rate limit by doing the following:  
1. Close Jupyter Notebook.  
2. Open your terminal/command prompt.  
3. Run the following command:  
```
jupyter notebook --NotebookApp.iopub_data_rate_limit=1000000000
```

That should have increased the data rate limit to 1 GB/s, and you can now return to working on your notebook.  

→ Note that if you are using Anaconda to run this Notebook, run that command on Anaconda's *CMD.exe Prompt*.

---
## Libraries Imported

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import display

---
## **Data Collection**  

### **Load CSV Data into DataFrame**

In [ ]:
# For file in the same directory
df = pd.read_csv('unclean_goodreads_library_export.csv') 

# If the CSV file is in a different directory, specify the path
# df = pd.read_csv('/path/to/your_file.csv')

# If the CSV file is hosted online, provide the URL
# df = pd.read_csv('https://example.com/your_file.csv')

### **Display original data**

In [ ]:
# Set display option to show all rows
pd.set_option('display.max_rows', None)

# Set display option to show all columns without truncation
pd.set_option('display.max_columns', None)

# Set display option to show ISBN13 without scientific notation
pd.options.display.float_format = '{:.0f}'.format

# Display all rows of the DataFrame
display(df)

---

## Initial Data Assessment

### **Display the structure of the dataset (columns, data types, dataframe size)**

In [ ]:
# Display columns and data types
print("Columns and Data Types:")
display(df.dtypes)

print("\nSize of the DataFrame:")
display(df.shape)

### **Change data types to match expectations**  

Date columns "**Date Read**" and "**Date Added**" should be in the DD/MM/YYYY formats.  

In [ ]:
# Convert Data Read and Data Added columns to DD/MM/YYYY
df['Date Read'] = pd.to_datetime(df['Date Read'], format='%d/%m/%Y', errors='coerce')
df['Date Added'] = pd.to_datetime(df['Date Added'], format='%d/%m/%Y', errors='coerce')


# Verify changes and check distribution of values
print("Data type of 'Date Read' column:", df['Date Read'].dtype, end='\n\n')
print("Data type of 'Date Added' column:", df['Date Added'].dtype, end='\n\n')

The column "**Spoiler**" refers to the existence of book spoilers in the "My Review" data. Currently the column marks TRUE when there is a value, and leaves it blank when there are no spoilers. To better analyse this, I'm changing the TRUE values to 1, and the empty cells to 0, thus converting the column to integer values, and then to boolean.

In [ ]:
# Fill missing values with 0
df['Spoiler'] = df['Spoiler'].fillna(0)

# Replace 'TRUE' values with 1
df.loc[df['Spoiler'] == 'TRUE', 'Spoiler'] = 1

# Convert to boolean
df['Spoiler'] = df['Spoiler'].astype(bool)

# Convert column to categorical type
df['Spoiler'] = df['Spoiler'].astype('category')

# Verify changes and check distribution of values
print("Data type of 'Spoiler' column after conversion:", df['Spoiler'].dtype)
print("\nDistribution of values in 'Spoiler' column:")
print(df['Spoiler'].value_counts())

**ISBN** and **ISBN13** are are unique identifiers and have no inherent order, hence why these should be classified as categorical data and not numeric.

In [ ]:
# Convert column to categorical type
df['ISBN'] = df['ISBN'].astype('category')
df['ISBN13'] = df['ISBN13'].astype('category')

# Verify changes and check distribution of values
print("Data type of 'ISBN' column after conversion:", df['ISBN'].dtype)
print("Data type of 'ISBN13' column after conversion:", df['ISBN13'].dtype)

### **List which collumns are Categorical and which are Numerical values**

In [ ]:
def classify_columns(df):
    categorical_cols = []
    numerical_cols = []
    
    for col in df.columns:
        if pd.api.types.is_categorical_dtype(df[col]) or pd.api.types.is_string_dtype(df[col]):
            categorical_cols.append(col)
        elif pd.api.types.is_numeric_dtype(df[col]):
            numerical_cols.append(col)
    
    return categorical_cols, numerical_cols

categorical_cols, numerical_cols = classify_columns(df)

print("Categorical columns:", categorical_cols, end='\n\n')
print("Numerical columns:", numerical_cols)

### **Check for missing data**

In [ ]:
# Calculate missing values
missing_values = df.isnull().sum()

# Create a DataFrame for missing values
missing_values_df = missing_values.to_frame().reset_index()
missing_values_df.columns = ['Column', 'Missing Values']

# Display the missing values DataFrame
display(missing_values_df)

### **Visualise missing values**

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), cmap='viridis', cbar=False)
plt.title('Missing Values Heatmap')
plt.show()

### **Check for duplicate entries**

In [ ]:
# Check for duplicate entries
duplicate_entries = df.duplicated()

# Display duplicate entries
print("Duplicate entries:")
display(df[duplicate_entries])

### **For categorical values, check for unique values and compare how many unique values are per column**

In [ ]:
for col in categorical_cols:
    unique_values = df[col].unique()
    num_unique = len(unique_values)
    total_values = len(df[col])
    print(f"Column: {col}")
    print(f"Number of unique values: {num_unique}")
    print(f"Total number of values: {total_values}", end='\n\n')
    
    # Uncomment the following to see all unique values
    # print(f"Unique values: {unique_values}\n")

### **For numerical values, check for unique values and compare how many unique values are per column**

In [ ]:
for col in numerical_cols:
    unique_values = df[col].unique()
    num_unique = len(unique_values)
    total_values = len(df[col])
    print(f"Column: {col}")
    print(f"Number of unique values: {num_unique}")
    print(f"Total number of values: {total_values}", end='\n\n')
    
    # Uncomment the following to see all unique values
    # print(f"Unique values: {unique_values}\n")

### **Check for Outliers and Analyse Statistics**  

The following boxplots and statistical analysis refer to columns where checking for outliers might be relevant.

In [ ]:
columns_to_check = ["My Rating", "Average Rating", "Number of Pages", "Year Published", "Original Publication Year"]

# Plot box plots for each column
for col in columns_to_check:
    plt.figure(figsize=(10, 2))
    sns.boxplot(x=df[col])
    
    # Calculate summary statistics
    desc = df[col].describe()
    min_val = desc['min']
    q1 = desc['25%']
    median = desc['50%']
    q3 = desc['75%']
    max_val = desc['max']
    
    # Add annotations to the box plot
    plt.text(1.05, 0.5, f"Min: {min_val:.2f}\nQ1: {q1:.2f}\nMedian: {median:.2f}\nQ3: {q3:.2f}\nMax: {max_val:.2f}",
             transform=plt.gca().transAxes, fontsize=10, verticalalignment='center', horizontalalignment='left')
    
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)
    plt.show()

### Check that the structure of the dataset (columns, data types, dataframe size) has all changes

In [ ]:
print("Columns and Data Types:")
display(df.dtypes)

print("\nSize of the DataFrame:")
display(df.shape)

## Data Cleaning  

### Remove duplicate data entries and confirm removal

In [ ]:
# Remove duplicate entries
df = df[~duplicate_entries]

# Confirm removal by checking the shape of the DataFrame
print("Shape after removing duplicates:", df.shape)

### Replace missing values with mode, median, NaN or Not assigned  

Fill with NaN:

- **Additional Authors**: Missing values here indicate the absence of additional authors, so filling with NaN is appropriate.
- **My Review**: If there is no review provided, it's logical to fill the missing values with NaN.
- **Private Notes**: Similar to My Review, missing values in this column likely indicate the absence of private notes.
  
Fill with Median:

- **Number of Pages**: Filling missing values with the median number of pages can help retain the central tendency of the data without introducing bias.
- **Original Publication Year**: Filling missing values with the median publication year can help retain the central tendency of the data without introducing bias.  

Fill with Mode:

- **Date Read**: If the date of reading is not provided, filling missing values with the mode (most frequent date read) could be appropriate.
- **Publisher**: If the publisher information is missing, filling missing values with the mode (most frequent publisher) could be appropriate.
- **Binding**: If the binding type is not provided, filling missing values with the mode (most frequent binding type) could be appropriate.  

Fill with Not assigned:

- **Bookshelves**: If a book is not assigned to any bookshelf, it could be considered as missing information. Hence filling missing values with a new category for not assigned could be appropriate.
- **Bookshelves with positions**: similar to the case above where filling the missing values with a new category for not assigned could be appropriate.

In [ ]:
# Fill with NaN
nan_columns = ['Additional Authors', 'My Review', 'Private Notes']
df[nan_columns] = df[nan_columns].fillna(pd.NA)

# Fill with median
median_columns = ['Number of Pages', 'Original Publication Year']
df[median_columns] = df[median_columns].fillna(df[median_columns].median())

# Fill with mode
mode_columns = ['Date Read', 'Publisher', 'Binding']
for col in mode_columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Fill with Not assigned
not_assigned_columns = ['Bookshelves', 'Bookshelves with positions']
df[not_assigned_columns] = df[not_assigned_columns].fillna('Not assigned')

# Display the cleaned DataFrame
display(df)

### Enconde categorical variables

- **Title**: One-Hot Encoding is suitable if the number of unique titles is relatively small and manageable. Each title would be represented by a binary vector, where each element corresponds to whether the title matches a particular category or not.

In [ ]:
# Perform one-hot enconding
title_encoded = pd.get_dummies(df['Title'], prefix='Title')

# Concatenate the encoded column with the original DF
df_encoded = pd.concat([df, title_encoded], axis=1)

# Drop the original 'Title' column if needed
# df_encoded.drop('Title', axis=1, inplace=True)

# Display the encoded DataFrame
print(df_encoded)

- **Author**: Target Encoding encodes each author based on the mean of the target variable (e.g., rating) for books written by that author. This can capture the relationship between author and target variable, making it useful for predictive modeling.

- **Author l-f**: Target Encoding, same rationale as Author.


- **Additional Authors**: Frequency Encoding since missing values here indicate the absence of additional authors, Frequency Encoding can be used to encode each additional author based on its frequency of occurrence in the dataset.

- **ISBN**: Frequency Encoding is preferred as there are many unique ISBN values.

- **ISBN13**: Frequency Enconding, same rationale as ISBN.

- **Publisher**: Target Encoding can capture the relationship between publisher and the target variable. 


- **Binding**: One-Hot Encoding since Binding types are nominal categories with no ordinal relationship, making One-Hot Encoding suitable for encoding each type as a binary feature.

- **Bookshelves**: Frequency Enconding as there many multiple unique categories.

- **Bookshelves with positions**: Frequency Encoding as there are multiple unique categories.

- **Exclusive Shelf**: One-Hot Encoding as it is a categorical variable with multiple categories.

- **My Review**: Frequency Encoding as reviews are not categorized.

- **Spoiler**: One-Hot Encoding as this is a boolean categorical variable. 

- **Private Notes**: Frequency Encoding as notes are not categorized. 